# **<font color=darkgreen>DB access**

## **Microsoft SQL server**  
Microsoft がテストを行い pyodbc ドライバーを推奨しています。 次のいずれかのドライバーを選択し、開発環境を構成します。  
+   Python SQL ドライバー - pyod　
+ Python SQL ドライバー - pymssql

ここでは、pyodbc を使用する。
Windows install 
Microsoft ODBC Driver 18 for SQL Server (x64) のダウンロード　https://go.microsoft.com/fwlink/?linkid=2280794

  pyodbc パッケージをインストールする  
  command prompt で
  `python -m pip install pyodbc` を実行する。  bc

In [1]:
#!python -m pip install pyodbc

In [9]:
"""
Connects to a SQL database using pyodbc
"""
import pyodbc
import pprint

#SERVER = '<server-address>'
#DATABASE = '<database-name>'
#USERNAME = '<username>'
#PASSWORD = '<password>'

SERVER = '172.31.12.11'
DATABASE = 'MOSAIQ'
USERNAME = 'MOSAIQUser'
PASSWORD = 'mosaiq'

#driver = "{ODBC Driver 18 for SQL Server}"
driver = "{SQL Server}"

connectionString = f'DRIVER={driver};' +f'SERVER={SERVER};' +\
                   f'DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};Trusted_Connection=no'
print("str=", connectionString)
conn = pyodbc.connect(connectionString)

# str= DRIVER={SQL Server};SERVER=172.31.12.11;DATABASE=MOSAIQ;UID=MOSAIQUser;
# PWD=mosaiq;Trusted_Connection=no

str= DRIVER={SQL Server};SERVER=172.31.12.11;DATABASE=MOSAIQ;UID=MOSAIQUser;PWD=mosaiq;Trusted_Connection=no


In [13]:
SQL_QUERY = """
SELECT TOP 5 c.CustomerID, c.CompanyName, COUNT(soh.SalesOrderID) AS OrderCount 
FROM SalesLT.Customer AS c 
LEFT OUTER JOIN SalesLT.SalesOrderHeader AS soh ON c.CustomerID = soh.CustomerID 
GROUP BY c.CustomerID, c.CompanyName 
ORDER BY OrderCount DESC;
"""
SQL_QUERY = '''select pat_id1,user_defined_dttm_1,user_defined_pro_id_3 from admin where pat_id1 = '16119';'''
#SQL_QUERY = '''select table_name from information_schema.tables where table_type='base table';'''
#SQL_QUERY = '''select top 100 pat_id1, last_name, first_name from patient ;'''
#SQL_QUERY = '''select * from prompt where pro_id='13112';'''
#SQL_QUERY = '''select pro_id,text from prompt where pro_id>='13100' and pro_id<='13200';'''

In [ ]:
# SQL 実行、結果を印字

def query(sql):
    conn = pyodbc.connect(connectionString)
    cursor = conn.cursor()
    
    cursor.execute(sql) # SQL 実行
    
    rows = cursor.fetchall()
    print(rows)
    cursor.close()
    conn.close()
    return rows

In [15]:
rows = query(SQL_QUERY)

pprint.pprint(rows)

# [(16119, datetime.datetime(2023, 4, 29, 0, 0), 13114)]
# 管理番号、最終生存確認日、病態

[(16119, datetime.datetime(2023, 4, 29, 0, 0), 13114)]
[(16119, datetime.datetime(2023, 4, 29, 0, 0), 13114)]


In [16]:
status = {'13111':'1.非担癌生存','13114':'4.原病死', '13113':'3.担癌不詳生存', '13112':'2.担癌生存','13115':'5.他病死',
          '13116':'6.不明死', '13117':'7.消息不明' }
st=rows[0][2]
print(st)
print(status[str(st)])

13114
4.原病死


In [17]:
# ID= 16119 の カルテ番号、氏名を表示
sql = '''
select i.ida,p.last_name,p.first_name from ident i 
  left outer join patient p on i.pat_id1=p.pat_id1 
  where i.pat_id1=16839 ;'''
print("SQL=", sql)
query(sql)

SQL= 
select i.ida,p.last_name,p.first_name from ident i 
  left outer join patient p on i.pat_id1=p.pat_id1 
  where i.pat_id1=16839 ;
[('', '足利', 'update')]


[('', '足利', 'update')]

In [ ]:
# 性別、生年月日 を表示
sql= """select i.Pat_Id1,i.ida,p.Last_Name,p.First_Name,a.Gender,
            dbo.fn_GetPatientAge(i.Pat_Id1,GETDATE()) from ident i
            left outer join patient p on i.Pat_Id1=p.Pat_ID1
            left outer join admin a on a.Pat_ID1=i.Pat_Id1
            where i.ida='00000202308' ;"""
print("SQL=", sql)
query(sql)

In [ ]:
# ID=16839 の病態、最終生存確認日 を表示
sql = '''select pat_id1,user_defined_dttm_1,user_defined_pro_id_3 
from admin where pat_id1 = '16839';'''

print("SQL=", sql)
query(sql)

SQL= select pat_id1,user_defined_dttm_1,user_defined_pro_id_3 
from admin where pat_id1 = '16839';
[(16839, None, 13114)]


[(16839, None, 13114)]

In [32]:
# ID=16421 の病態、最終生存確認日 UPDATE
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

sql = '''update admin set user_defined_pro_id_3 = ?
where pat_id1 = ? ;'''
values=(13114, 16839) # 現病死、 ID=16421
print("SQL=", sql)
print("values=", values)
try:
    cursor.execute(sql, values)
    conn.commit()
except Exception as e:
    print(f'error: {e}')
    conn.rollback()
finally:
    cursor.close()
    conn.close()
    print("実行しました.")


SQL= update admin set user_defined_pro_id_3 = ?
where pat_id1 = ? ;
values= (13114, 16839)
実行しました.


In [ ]:
# transaction
def trans(sql, values):
    print("sql=", sql)
    print("values=", values)
    
    try:
        # 接続
        conn = pyodbc.connect(connectionString)
        cursor = conn.cursor()
    
        # トランザクション開始
        #cursor.execute("BEGIN TRANSACTION")
    
        # SQL文の実行
        cursor.execute(sql, values)
    
        # コミット
        conn.commit()
        print("データが更新されました")
    
    except pyodbc.Error as ex:
        print('Error:', ex)
        # ロールバック
        conn.rollback()
    
    finally:
        # 接続を閉じる
        if conn:
            conn.close()
    return
#---
sql='''update admin set user_defined_pro_id_3 = ?
where pat_id1 = ? ;'''
values=(13113, 16839) 
trans(sql, values)

#---

sql= update admin set user_defined_pro_id_3 = ?
where pat_id1 = ? ;
values= (13113, 16839)
データが更新されました


# <font color=red>**test**

In [37]:
# transaction
def trans2(sql):
    print("### trans2 ###")
    print("sql=", sql)
    print("values=", values)
    
    try:
        # 接続
        conn = pyodbc.connect(connectionString)
        cursor = conn.cursor()
    
        # トランザクション開始
        #cursor.execute("BEGIN TRANSACTION")
    
        # SQL文の実行
        cursor.execute(sql)
    
        # コミット
        conn.commit()
        print("データが更新されました")
    
    except pyodbc.Error as ex:
        print('Error:', ex)
        # ロールバック
        conn.rollback()
    
    finally:
        # 接続を閉じる
        if conn:
            conn.close()
#---
sql='''update admin set user_defined_pro_id_3 = 13114
where pat_id1 = 16839 ;'''
#values=(13113, 16839) 
trans2(sql)

#---

### trans2 ###
sql= update admin set user_defined_pro_id_3 = 13114
where pat_id1 = 16839 ;
values= (13113, 16839)
データが更新されました


In [1]:
# すべてのセルを文字列として取得
import nbformat

# 現在開いているノートブックを読み込む
with open('3a_db_access.ipynb', 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# すべてのセルを文字列として取得
all_code = []
for cell in nb.cells:
    if cell.cell_type == 'code':
        all_code.append(cell.source)
# テキストファイルに保存
with open('3a_db_access.py', 'w') as f:
    for code in all_code:
        f.write(code)
print("END")

END


In [ ]:
!jupyter nbconvert --to python 3a_db_access.ipynb --output output.py